In [12]:
import pandas as pd

names = pd.read_csv("/Users/kritinshanmugam/Desktop/names.csv", header=None)
names_list = names.iloc[:, 0].tolist()

X = pd.read_csv("/Users/kritinshanmugam/Desktop/genes.csv", header=None, names=names_list)
y = pd.read_csv("/Users/kritinshanmugam/Desktop/labels.csv", header=None, names = ['Label'])

In [13]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np

label_encoder = LabelEncoder()

# Handle missing values
X = X.dropna()
y = y.dropna()

# Standardize features
X = (X - X.mean()) / X.std()

# Create copy of original (untransformed) y data to compare later with predictions
y_orig = y.copy()

print(X.head())

   0610007P14Rik  0610009B22Rik  0610009L18Rik  0610009O20Rik  0610010F05Rik  \
0       6.357779      -0.129192      -0.133012      -0.339486      -0.441585   
1      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   
2      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   
3      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   
4      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   

   0610012D04Rik  0610012G03Rik  0610025J13Rik  0610030E20Rik  0610031O16Rik  \
0      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
1      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
2      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
3      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
4      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   

   ...    mt-Co2    mt-Co3   mt-Cytb  

In [14]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

'''
# Create list to store incorrect predictions
xg_incorrect = []

for row in range(len(xg_preds)):
    if xg_preds.iloc[row, 0] != y_test.iloc[row, 0]:
        xg_incorrect.append(row)
        
print(len(xg_incorrect))
'''

# List to store each XGBoost model
models = []



# Create 100 models, train on 90% of dataset, find model with lowest score
for i in range(100):
    
    print(i)
    
    # Create temporary XGBoost instance
    temp_model = XGBClassifier()
    
    # Split into 90% training, 10% test
    t_x_train, t_x_test, t_y_train, t_y_test = train_test_split(X, y, test_size=0.1)
    
    t_y_test = pd.DataFrame(t_y_test)
    actual = t_y_test.values
    
    # Train the model
    temp_model.fit(t_x_train, t_y_train)
    
    # Predict based on x_test
    temp_pred = temp_model.predicit(t_x_test)
    temp_pred = pd.DataFrame(temp_pred)
    
    # Get F1 score
    temp_f1 = f1_score(actual, temp_pred.values)
    
    # Add the model to the list for storage
    models.append((temp_model, temp_f1))
    
print(models)


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x1a327c6d0>>
Traceback (most recent call last):
  File "/Users/kritinshanmugam/anaconda3/lib/python3.11/site-packages/xgboost/core.py", line 589, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Get the actual values and predicted values in array-like format
actual = y_test.values

predicted = xg_preds.values

# Generate a confusion matrix
confusion = confusion_matrix(actual, predicted, labels=xg_classifier.classes_) 

# Generate display for confusion matrix
display = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=xg_classifier.classes_)

display.plot()

In [ ]:
import shap

explainer = shap.TreeExplainer(classifier, x_test)
shap_values = explainer(x_test)

for row in incorrect_list:
    shap.plots.waterfall(shap_values[row])